In [1]:
import requests
import re
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd

In [2]:
main_link = "https://www.car.gr/classifieds/cars/?fs=1&pg=%s"

In [3]:
# string formatting
max_pages= 3
for page_number in range(1, max_pages+1):
    print(main_link % page_number)

https://www.car.gr/classifieds/cars/?fs=1&pg=1
https://www.car.gr/classifieds/cars/?fs=1&pg=2
https://www.car.gr/classifieds/cars/?fs=1&pg=3


In [4]:
# string formatting
for page_number in range(1, max_pages+1):
    page_link = f"https://www.car.gr/classifieds/cars/?fs=1&pg={page_number}"
    print(page_link)

https://www.car.gr/classifieds/cars/?fs=1&pg=1
https://www.car.gr/classifieds/cars/?fs=1&pg=2
https://www.car.gr/classifieds/cars/?fs=1&pg=3


In [2]:
html_pages = []
for page_number in range(1, max_pages+1):
    print(main_link % page_number)
    html = requests.get(main_link % page_number)
    html_pages.append(html)
    
html_pages

NameError: name 'max_pages' is not defined

In [1]:
html_pages_content = [html.content for html in html_pages]
#html_pages_content

NameError: name 'html_pages' is not defined

In [3]:
html = html_pages_content[0]
#html

NameError: name 'html_pages_content' is not defined

In [4]:
soup = BeautifulSoup(html, "html.parser")
#soup

NameError: name 'BeautifulSoup' is not defined

In [9]:
soup.find("span", class_="total-results-num-text").text

'\n            (95.089 αγγελίες)\n          '

In [10]:
# Number of total results
num_results = 0
for f in soup.find_all("span", class_="total-results-num-text"):
    if "αγγελίες" in str(f).lower():
        if (
            re.findall("\d{1,3}\.*\d{0,3} αγγελίες", str(f).lower())[0]
            .split()[0]
            .replace(".", "")
            .isdigit()
        ):
            num_results = int(
                re.findall("\d{1,3}\.*\d{0,3} αγγελίες", str(f).lower())[0]
                .split()[0]
                .replace(".", "")
            )
num_results

95089

In [11]:
# Number of page results
len(soup.find_all("a", class_="row-anchor"))

25

In [12]:
soup.find_all("div", class_="price-tag")[0].find('span').find('span').text

# <div class="price-tag current-price" data-v-c73186fe="">

'9.900'

In [14]:
# Attributes
soup.find("h2", class_="title")

<h2 class="title mr-0" data-v-08c82b20="" target="_self" title="Citroen C4 '18 NEW 1.6 BLUEHDI 100 S&amp;S LIVE ΕΛΛΗΝΙΚΟ">
  Citroen C4 '18 NEW 1.6 BLUEHDI 100 S&amp;S LIVE ΕΛΛΗΝΙΚΟ
</h2>

In [16]:
soup.find("h2", class_="title").attrs

{'title': "Citroen C4 '18 NEW 1.6 BLUEHDI 100 S&S LIVE ΕΛΛΗΝΙΚΟ",
 'target': '_self',
 'class': ['title', 'mr-0'],
 'data-v-08c82b20': ''}

In [17]:
# parse prices
prices = [
            int(item.find('span').find('span').text.replace(".", ""))
            for item in soup.find_all("div", class_="price-tag")
            if item.find('span').find('span').text != "Ρωτήστε"
        ]
prices

[9900,
 58000,
 58900,
 22800,
 15900,
 30990,
 6200,
 23500,
 13990,
 21500,
 1500,
 20999,
 26990,
 8700,
 17500,
 13850,
 19990,
 15500,
 14990,
 14990,
 1700,
 16990,
 6900,
 16990,
 14500]

In [5]:
# Ads list
ads_list = soup.find_all("div", class_="search-row swipable")
#ads_list

NameError: name 'soup' is not defined

In [19]:
# Use 1st ad
car_item = ads_list[0]

car_id = int(
                car_item.find("a", class_="row-anchor")
                .get("href")
                .replace("/", "")
                .split("-")[0]
                .replace("classifiedscarsview", "")
            )
car_id

327049197

In [20]:
# title
title = str(car_item.find("h2", class_="title").text).strip()
title

"Citroen C4 '18 NEW 1.6 BLUEHDI 100 S&S LIVE ΕΛΛΗΝΙΚΟ"

In [21]:
# price
price = car_item.find("div", class_="price-tag").span.span.text
price

'9.900'

In [22]:
# url
url = car_item.find("a", class_="row-anchor").get("href")
url

'/classifieds/cars/view/327049197-citroen-c4?from-promotion=1'

In [23]:
all_ads = []
for car_item in ads_list:
    car_id = int(
                car_item.find("a", class_="row-anchor")
                .get("href")
                .replace("/", "")
                .split("-")[0]
                .replace("classifiedscarsview", "")
            )
    
    title = str(car_item.find("h2", class_="title").text.strip()
    
    price = car_item.find("div", class_="price-tag").span.span.text
    
    url = car_item.find("a", class_="row-anchor").get("href")
    
    data = [car_id, title, price, url]
    
    all_ads.append(data)
    
car_ads = pd.DataFrame(all_ads, columns = ['car_id', 'title', 'price', 'url'])    
car_ads          

,car_id,title,price,url
0,327049197,Citroen C4 '18 NEW 1.6 BLUEHDI 100 S&S LIVE ΕΛ...,9.900,/classifieds/cars/view/327049197-citroen-c4?fr...
1,327415429,Porsche Cayenne '15 e hybrid,58.000,/classifieds/cars/view/327415429-porsche-cayenne
2,326754981,Land Rover Range Rover Sport '15 HSE HYBRID,58.900,/classifieds/cars/view/326754981-land-rover-ra...
3,319098450,Bmw 730 '06,22.800,/classifieds/cars/view/319098450-bmw-730
4,321367575,Mercedes-Benz E 200 '09 Compressor Elegance,15.900,/classifieds/cars/view/321367575-mercedes-benz...
5,327823305,Honda CR-V '16 EXECUTIVE AUTOMATIC PANORAMA,30.990,/classifieds/cars/view/327823305-honda-cr-v
6,327864581,Chevrolet Spark '13 ΕΥΚΑΙΡΙΑ!!!!!,6.200,/classifieds/cars/view/327864581-chevrolet-spark
7,318693441,Ford Kuga '17 ST-Line Αυτόματο Full Extra,23.500,/classifieds/cars/view/318693441-ford-kuga
8,327312224,Renault Clio '16 GT Line Bose Navi Camera,13.990,/classifieds/cars/view/327312224-renault-clio
9,327103226,Honda HR-V '15 EXECUTIVE PANORAMA,21.500,/classifieds/cars/view/327103226-honda-hr-v
